In [1]:
import os
import json
from fnmatch import fnmatch
from math import pi
import pandas as pd
import cv2 as cv
import argparse
import os
import sys
from itertools import permutations
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [2]:
def fddb_dataset(annotations):
    with open(annotations) as f:
        imgPath = [] 
        rad_x = []
        rad_y = []
        angle = []
        center_x = []
        center_y = []
        detection_score = []
        lineId = 0
        lines = f.readlines()
        while lineId < len(lines):
            Path = lines[lineId]
            lineId += 1
            # Faces
            numFaces = int(lines[lineId])
            for i in range(numFaces):
                lineId += 1
                params = [float(v) for v in lines[lineId].split()]
                radx, rady, ang, centerx, centery, score = params
                imgPath.append(Path) 
                rad_x.append(radx)
                rad_y.append(rady)
                angle.append(ang)
                center_x.append(centerx)
                center_y.append(centery)
                detection_score.append(score)
                
            lineId += 1
        data = pd.DataFrame(
                {'imgPath': imgPath,
                 'rad_x': rad_x,
                 'rad_y': rad_y,
                 'angle': angle,
                 'center_x': center_x,
                 'center_y': center_y,
                })
            
        return data

In [3]:
data = fddb_dataset("C:\\Users\\nidhimh\\Downloads\\FDDB-folds\\FDDB-fold-01-ellipseList.txt")


In [4]:
data

,imgPath,rad_x,rad_y,angle,center_x,center_y
0,2002/08/11/big/img_591\n,123.583300,85.549500,1.265839,269.693400,161.781200
1,2002/08/26/big/img_265\n,67.363819,44.511485,-1.476417,105.249970,87.209036
2,2002/08/26/big/img_265\n,41.936870,27.064477,1.471906,184.070915,129.345601
3,2002/08/26/big/img_265\n,70.993052,43.355200,1.370217,340.894300,117.498951
4,2002/07/19/big/img_423\n,87.080955,59.379319,1.550861,255.383099,133.767857
...,...,...,...,...,...,...
510,2002/07/29/big/img_889\n,58.056741,41.854608,1.280160,198.496421,86.716075
511,2002/08/18/big/img_188\n,67.995400,38.216200,-1.559920,208.966471,109.764400
512,2002/08/07/big/img_1453\n,58.887348,37.286244,1.441974,88.083450,78.409537
513,2002/08/07/big/img_1453\n,60.381076,40.303691,1.377522,260.502940,102.769525


In [5]:
key = ['imgPath']

In [6]:
series = (data.groupby(key, sort=False)[data.columns.difference(key)].apply(lambda x: x[['rad_x', 'rad_y', 'angle', 'center_x', 'center_y']].to_dict('records')))

In [7]:
series

imgPath
2002/08/11/big/img_591\n      [{'rad_x': 123.5833, 'rad_y': 85.5495, 'angle'...
2002/08/26/big/img_265\n      [{'rad_x': 67.363819, 'rad_y': 44.511485, 'ang...
2002/07/19/big/img_423\n      [{'rad_x': 87.080955, 'rad_y': 59.379319, 'ang...
2002/08/24/big/img_490\n      [{'rad_x': 54.692105, 'rad_y': 35.056825, 'ang...
2002/08/31/big/img_17676\n    [{'rad_x': 37.099961, 'rad_y': 29.0, 'angle': ...
                                                    ...                        
2002/08/14/big/img_1065\n     [{'rad_x': 89.827412, 'rad_y': 61.125968, 'ang...
2002/07/29/big/img_889\n      [{'rad_x': 50.490436, 'rad_y': 33.977237, 'ang...
2002/08/18/big/img_188\n      [{'rad_x': 67.9954, 'rad_y': 38.2162, 'angle':...
2002/08/07/big/img_1453\n     [{'rad_x': 58.887348, 'rad_y': 37.286244, 'ang...
2002/08/02/big/img_760\n      [{'rad_x': 134.444208, 'rad_y': 92.088647, 'an...
Length: 290, dtype: object

In [8]:
series.to_json('C:\\Users\\nidhimh\\Documents\\MaskRCNN\\dataset\\train\\fullAnno.json')

In [9]:
annotations = json.load(open('C:\\Users\\nidhimh\\Documents\\MaskRCNN\\dataset\\train\\fullAnno.json'))
annotations

{'2002/08/11/big/img_591\n': [{'rad_x': 123.5833,
   'rad_y': 85.5495,
   'angle': 1.265839,
   'center_x': 269.6934,
   'center_y': 161.7812}],
 '2002/08/26/big/img_265\n': [{'rad_x': 67.363819,
   'rad_y': 44.511485,
   'angle': -1.476417,
   'center_x': 105.24997,
   'center_y': 87.209036},
  {'rad_x': 41.93687,
   'rad_y': 27.064477,
   'angle': 1.471906,
   'center_x': 184.070915,
   'center_y': 129.345601},
  {'rad_x': 70.993052,
   'rad_y': 43.3552,
   'angle': 1.370217,
   'center_x': 340.8943,
   'center_y': 117.498951}],
 '2002/07/19/big/img_423\n': [{'rad_x': 87.080955,
   'rad_y': 59.379319,
   'angle': 1.550861,
   'center_x': 255.383099,
   'center_y': 133.767857}],
 '2002/08/24/big/img_490\n': [{'rad_x': 54.692105,
   'rad_y': 35.056825,
   'angle': -1.384924,
   'center_x': 145.665694,
   'center_y': 78.101005}],
 '2002/08/31/big/img_17676\n': [{'rad_x': 37.099961,
   'rad_y': 29.0,
   'angle': 1.433107,
   'center_x': 28.453831,
   'center_y': 37.664572},
  {'rad_x': 7

In [10]:
#to count instances of ellipses in each image
for a in annotations:
    countdict = 0
    for i in annotations[a]:
        if type(i) is dict:
            countdict += 1
    print(countdict)

1
3
1
1
2
2
3
2
1
1
1
6
2
2
1
1
2
2
2
2
2
2
1
3
1
1
2
2
1
1
1
3
2
1
1
1
1
1
2
2
1
7
1
1
1
1
1
1
1
2
1
1
1
2
1
1
1
1
1
2
1
1
7
4
1
1
2
6
3
2
2
5
2
1
1
1
2
3
1
7
1
2
2
5
1
1
2
1
1
2
2
1
1
1
2
1
2
1
1
4
1
1
1
1
2
1
3
1
1
1
2
1
1
1
1
1
1
3
4
3
1
3
1
3
1
1
1
3
1
1
2
4
1
1
1
1
5
1
1
1
1
1
1
3
2
2
1
1
6
6
1
2
1
1
1
1
2
1
2
1
3
1
3
2
1
5
2
1
1
4
1
1
3
1
3
2
1
10
1
1
1
1
3
1
1
2
1
2
1
2
1
1
3
4
1
1
2
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
2
2
2
1
1
1
1
2
2
1
1
2
1
7
6
1
1
1
1
5
1
2
1
1
4
1
1
1
2
5
1
1
3
3
1
2
2
2
1
3
1
1
1
1
3
1
1
1
1
1
1
1
10
1
2
3
1
2
1
1
1
1
1
1
3
1
2
1
1
1
1
1
1
1
2
1
2
1
